In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import numpy as np

from torch.utils.data import Dataset, DataLoader

  Using cached https://github.com/ufoym/imbalanced-dataset-sampler/archive/master.zip
  Created wheel for torchsampler: filename=torchsampler-0.1.1-py3-none-any.whl size=3828 sha256=d3a493c4403cecab0ad5cc378d6b69f14dc7b41d279926377aa472e97a6ec489
  Stored in directory: /tmp/pip-ephem-wheel-cache-nfbbu58i/wheels/90/7e/cd/4f5ece8831ffd9a54a62db046bca608f3a0a514dc47cba0eea
Successfully built torchsampler


In [8]:
device = 'cuda:0'

model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True).to(device)
model.classifier[1] = nn.Linear(1280, 2).to(device)

Using cache found in /home/tannin/.cache/torch/hub/pytorch_vision_v0.10.0


In [2]:
def make_weights_for_balanced_classes(dataset, nclasses):                        
    count = [0] * nclasses                                 

    for item in dataset:     

        if len(item) == 1:
            print(item)        
        count[item[1]] += 1                                                     
    weight_per_class = [0.] * nclasses                                      
    N = float(sum(count))                                                   
    for i in range(nclasses):                                                   
        weight_per_class[i] = N/float(count[i])                                 
    weight = [0] * len(dataset)                                              
    for idx, val in enumerate(dataset):                                          
        weight[idx] = weight_per_class[val[1]]                                  
    return weight    


In [3]:
from data.plant_vilage.util import download_plantvillage_dataset, get_train_valid_test_dataset


train, val, test = get_train_valid_test_dataset('./plant-village', image_size=64)

weights = make_weights_for_balanced_classes(train, 2)
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))

In [15]:
train_loader = DataLoader(train, batch_size=128, sampler=sampler)
val_loader = DataLoader(val, batch_size=128)
test_loader = DataLoader(test, batch_size=128)

In [16]:
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

model.features.requires_grad_(False)
model.classifier.requires_grad_(True)

def validate_model(model, loader, criterion):
    """Validate the model"""

    # Set the model to evaluation mode.
    model.eval()

    # Initialize the loss and accuracy.
    loss = 0
    accuracy = 0

    # For each batch in the validation set...
    for batch_idx, (data, target) in enumerate(loader):
        # Send the batch to the device.

        data, target = data.to(device), target.to(device)

        # Expand image to have 3 channels.

        data = data.expand(data.shape[0], 3, data.shape[2], data.shape[3])

        # Upscale to 32x32

        data = F.interpolate(data, size=(32, 32))

        # Forward pass.
        with torch.no_grad():
            output = model(data)

        # Calculate the loss.
        loss += criterion(output, target).item()

        # Get the predictions.
        _, preds = torch.max(output, 1)

        # Calculate the accuracy.
        accuracy += torch.sum(preds == target).item()

    # Calculate the average loss and accuracy.
    loss /= len(loader) * 128
    accuracy /= len(loader) * 128

    return loss, accuracy


def train_model(model, train_loader, val_loader, optimizer, criterion, epochs=10):
    """Trains model"""

    # Put the model in training mode.
    model.train()

    # For each epoch...
    for epoch in range(epochs):
        # For each batch in the training set...
        for batch_idx, (data, target) in enumerate(train_loader):
            # Send the data and labels to the device.
            data, target = data.to(device), target.to(device)

            # Expand image to have 3 channels.

            data = data.expand(data.shape[0], 3, data.shape[2], data.shape[3])

            # Upscale to 32x32

            data = F.interpolate(data, size=(32, 32))


            # Zero out the gradients.
            optimizer.zero_grad()

            # Forward pass.
            output = model(data)

            # Calculate the loss.
            loss = criterion(output, target)

            # Backward pass.
            loss.backward()

            # Update the weights.
            optimizer.step()

            # Print the loss.
            if batch_idx % 100 == 0:
                print('Epoch: {}/{}'.format(epoch + 1, epochs),
                      'Loss: {:.4f}'.format(loss.item()))

        # Validate the model.
        val_loss, val_acc = validate_model(model, val_loader, criterion)

        # Print the validation loss.
        print('Validation Loss: {:.4f}'.format(val_loss))

        # Print the validation accuracy.
        print('Validation Accuracy: {:.4f}'.format(val_acc))

    # Test the model.
    test_loss, test_acc = validate_model(model, test_loader, criterion)

    # Print the test loss.
    print('Test Loss: {:.4f}'.format(test_loss))

    # Print the test accuracy.
    print('Test Accuracy: {:.4f}'.format(test_acc))

In [ ]:
# Freeze/unfreeze layers after converging with training

model.features[0:13].requires_grad_(False)
model.features[13:].requires_grad_(True)

In [21]:
train_model(model, train_loader, val_loader, optimizer, criterion, epochs=1)

Epoch: 1/1 Loss: 0.0440
Epoch: 1/1 Loss: 0.0523
Epoch: 1/1 Loss: 0.0417
Validation Loss: 0.0008
Validation Accuracy: 0.9619
Test Loss: 0.0008
Test Accuracy: 0.9506


In [33]:
# Save model state dict to file

torch.save(model.state_dict(), './plant-village-64.pt')

In [27]:
_train_loader = DataLoader(train, batch_size=128)

In [28]:
from sklearn.metrics import confusion_matrix

y_preds = []
y_trues = []

model.eval()

with torch.no_grad():
    for i, (data, target) in enumerate(_train_loader):
        y_preds.append(target.cpu())

        data, target = data.to(device), target.to(device)

        data = data.expand(data.shape[0], 3, data.shape[2], data.shape[3])

        data = F.interpolate(data, size=(32, 32))

        output = model(data)

        _, preds = torch.max(output, 1)

        y_trues.append(preds.cpu())
    
    for i, (data, target) in enumerate(val_loader):
        y_preds.append(target.cpu())

        data, target = data.to(device), target.to(device)

        data = data.expand(data.shape[0], 3, data.shape[2], data.shape[3])

        data = F.interpolate(data, size=(32, 32))

        output = model(data)

        _, preds = torch.max(output, 1)

        y_trues.append(preds.cpu())

    for i, (data, target) in enumerate(test_loader):
        y_preds.append(target.cpu())

        data, target = data.to(device), target.to(device)

        data = data.expand(data.shape[0], 3, data.shape[2], data.shape[3])

        data = F.interpolate(data, size=(32, 32))

        output = model(data)

        _, preds = torch.max(output, 1)

        y_trues.append(preds.cpu())

In [29]:
y_preds = np.concatenate(y_preds)
y_trues = np.concatenate(y_trues)

In [30]:
C = confusion_matrix(y_trues, y_preds)

In [31]:
C

array([[14805,   667],
       [  279, 38554]])

In [32]:
14805 / (14805 + 667)

0.9568898655635988